In [1]:
from copy import deepcopy
from src.sim import Scenario
from src.trades import Swap, Cycle
from src.configs import TOKEN_DTOs
from src.utils import get_crvusd_index
from src.agents.arbitrageur import optimize_cycle

In [2]:
scenario = Scenario("baseline", "weth")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()
scenario.prepare_for_trades(sample)

[INFO][16:53:17][src.sim.scenario]-75793: Fetching sim_market from subgraph.
[INFO][16:53:41][src.sim.scenario]-75793: Using 168480 1Inch quotes from 2023-12-07 15:53:29.509202 to 2023-12-14 15:53:29.509202
[INFO][16:53:41][src.utils.poolgraph]-75793: Found 20 valid cycles of length 3.


In [3]:
llamma = scenario.llamma
i = 0
j = 1
size = llamma.get_max_trade_size(i, j)

swap = Swap(llamma, i, j, None)

with llamma.use_snapshot_context():
    _, out_snapshot, _ = llamma.trade(i, j, size)

before = llamma.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = llamma.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [4]:
spool = scenario.stableswap_pools[0]
i = 0
j = 1
size = spool.get_max_trade_size(i, j)

swap = Swap(spool, i, j, None)

with spool.use_snapshot_context():
    _, out_snapshot, _ = spool.trade(i, j, size)

before = spool.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = spool.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [5]:
pool = deepcopy(scenario.stableswap_pools[0])
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [6]:
pool = deepcopy(scenario.llamma)
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [7]:
llamma = deepcopy(scenario.llamma)
spool = deepcopy(scenario.stableswap_pools[2])

trade3_token_in = TOKEN_DTOs[spool.coin_addresses[0]]
trade3_token_out = TOKEN_DTOs[llamma.COLLATERAL_TOKEN.address]
key = tuple(sorted([trade3_token_in, trade3_token_out]))
i3, j3 = key.index(trade3_token_in), key.index(trade3_token_out)
market = scenario.markets[key]

trade1 = Swap(llamma, 1, 0, None)  # weth -> crvusd
trade2 = Swap(spool, 1, 0, None)  # crvusd -> stablecoin
trade3 = Swap(market, i3, j3, None)

cycle = Cycle([trade1, trade2, trade3])

In [8]:
amt, profit = optimize_cycle(cycle, sample)
profit_ = cycle.populate(amt) * sample.prices_usd[cycle.basis_address]
assert profit == profit_

In [9]:
# TODO Test execute

In [ ]:
# TODO Test for liquidations too